# Exercise - Decision Tree

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['readmitted']
test_target = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [6]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['emergency_visit'] = np.where(df1['number_emergency'] > 0, 1, 0)
    
    return df1[['emergency_visit']]
    # You can use this to check whether the calculation is made correctly:
    #return df1

In [8]:
#Let's test the new function:

# Send train set to the function we created
new_col(train_inputs)

,emergency_visit
4166,0
5546,0
2957,0
6329,0
565,0
...,...
5734,0
5191,0
5390,0
860,0


##  Identify the numeric, binary, and categorical columns

In [9]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [10]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [11]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [12]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [13]:
feat_eng_columns = ['emergency_visit']

# Pipeline

In [16]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [17]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [18]:
# Create a pipeline for the transformed column here

my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col))])


In [20]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [21]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x64 sparse matrix of type '<class 'numpy.float64'>'
	with 84924 stored elements in Compressed Sparse Row format>

In [22]:
train_x.shape

(6066, 64)

# Tranform: transform() for TEST

In [23]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x64 sparse matrix of type '<class 'numpy.float64'>'
	with 36397 stored elements in Compressed Sparse Row format>

In [24]:
test_x.shape

(2600, 64)

# Calculate the Baseline

In [25]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5313221233102539


In [28]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365384615384615


# Train a DT model

In [29]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

In [30]:
tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=5)

### Calculate the accuracy

In [31]:
from sklearn.metrics import accuracy_score


In [32]:
#Train accuracy:
train_target_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_target_pred))


0.6252884932410155


In [33]:
#Test accuracy:
test_target_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.6038461538461538


# Train another DT model (with different parameters)

In [87]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=4)

In [88]:
tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=4)

### Calculate the accuracy

In [89]:
from sklearn.metrics import accuracy_score

In [90]:
#Train accuracy:
train_target_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_target_pred))


0.6147378832838774


In [91]:
#Test accuracy:
test_target_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.6146153846153846


# Optional: try grid search

In [65]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=5, high=20), 
              'min_samples_leaf': randint(low=5, high=20)}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=15, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

tree_gs.fit(train_x, train_target)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=15,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000272AC893EB0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000272AC893D60>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [66]:
cvres = tree_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.568246929542803 {'max_depth': 14, 'min_samples_leaf': 12}
0.6022110823030568 {'max_depth': 6, 'min_samples_leaf': 5}
0.6040254464607064 {'max_depth': 6, 'min_samples_leaf': 11}
0.5952867819924459 {'max_depth': 8, 'min_samples_leaf': 14}
0.5809431325386294 {'max_depth': 9, 'min_samples_leaf': 16}
0.5850644650009305 {'max_depth': 11, 'min_samples_leaf': 7}
0.5629715696647113 {'max_depth': 14, 'min_samples_leaf': 6}
0.5557192740370316 {'max_depth': 17, 'min_samples_leaf': 11}
0.5791291758285787 {'max_depth': 9, 'min_samples_leaf': 18}
0.5565417749232301 {'max_depth': 19, 'min_samples_leaf': 5}
0.6008924460573333 {'max_depth': 5, 'min_samples_leaf': 13}
0.5975921205067019 {'max_depth': 9, 'min_samples_leaf': 5}
0.5761610558868708 {'max_depth': 11, 'min_samples_leaf': 13}
0.5651166454567556 {'max_depth': 15, 'min_samples_leaf': 5}
0.5713818313683041 {'max_depth': 14, 'min_samples_leaf': 18}


In [67]:
#Find the best parameter set
tree_gs.best_params_

{'max_depth': 6, 'min_samples_leaf': 11}

In [68]:
tree_gs.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=11)

In [69]:
#Train accuracy:
train_target_pred = tree_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_target_pred))

0.6246290801186943


In [70]:
#Test accuracy:
test_target_pred = tree_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.6026923076923076
